In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import re
from tokenizer import split_into_sentences
import pos
import time
from tqdm.auto import tqdm
tqdm.pandas()

/Users/starspace/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/starspace/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/starspace/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/starspace/anaconda3/lib/python3.7/site-packages/tensorflow/pyth

In [2]:
# Initialize the tagger
tagger = pos.Tagger(
    model_file="tagger-v2.0.0.pt",
    device="cpu",
)

In [3]:
acc_words = '[Uu]m(fram|hverfis)|UM(FRAM|HVERFIS)|[Uu]m|UM|[Gg]egnum|GEGNUM|[Kk]ringum|KRINGUM|[Vv]ið|VIÐ|[Íí]|[Áá]'
dat_words = '[Ff]rá|FRÁ|[Aa][ðf]|A[ÐF]|[Áá]samt|ÁSAMT|[Gg]agnvart|GAGNVART|[Gg]egnt?|GEGNT?|[Hh]anda|HANDA|[Hh]já|HJÁ|[Mm]eð( )?fram|MEÐ( )?FRAM|[Mm]óti?|MÓTI?|[Uu]ndan|UNDAN|[Nn]álægt|NÁLÆGT'
gen_words = '[Tt]il|TIL|[Aa]uk|AUK|[Áá]n|ÁN|[Hh]andan|HANDAN|[Ii]nnan|INNAN|[Mm]eðal|MEÐAL|[Mm]egin|MEGIN|[Mm]ill(i|um)|MILL(I|UM)|[Oo]fan|OFAN|[Ss]akir|SAKIR|[Ss]ökum|SÖKUM|[Uu]tan|UTAN|[Vv]egna|VEGNA'
accdat_words = "[Ee]ftir|EFTIR|[Ff]yrir|FYRIR|[Mm]eð|MEÐ|[Uu]ndir|UNDIR|[Vv]ið|VIÐ|[Yy]fir|YFIR"
accgen_words = acc_words + "|" + gen_words
accdat_words_comb = acc_words + "|" + dat_words + "|" + accdat_words
accdatgen_words_comb = accdat_words_comb + "|" + gen_words

amounts = "([Hh]undr[au]ð|HUNDR[AU]Ð|[Þþ]úsund|ÞÚSUND|[Mm]illjón(ir)?|MILLJÓN(IR)?)"
date_ptrn = r"^((([012]?[1-9]|3[01])\. ?)?(jan(úar)?|feb(rúar)?|mars?|apr(íl)?|maí|jú[nl]í?|ág(úst)?|sep(t(ember)?)?|okt(óber)?|nóv(ember)?|des(ember)?) )\d{2,4}$"

In [4]:
sport_df = pd.read_csv('dataframe_sport_0503.csv', keep_default_na=False, na_values=['_'])

In [ ]:
other_df = pd.read_csv('dataframe_other_0503.csv', keep_default_na=False, na_values=['_'])

In [5]:
sport_df_use = sport_df[['sentence_id', 'token_id', 'before']]

In [ ]:
other_df_use = other_df[['sentence_id', 'token_id', 'before']]

In [6]:
""" get preceding and following words in a dataframe """
def make_dataframe_for_abbr(df, order, col):
    df['sentbefore'] = [""] + list(df[col][:-1])
    df['sent2before'] = ["", ""] + list(df[col][:-2])
    df['sent3before'] = ["", "", ""] + list(df[col][:-3])
    df['sentafter'] = list(df[col][1:]) + [""]
    if order == 'first':
        df['after'] = df['before']
        df['type'] = ""
    elif order == 'second':
        pass
    return df

In [7]:
sport_dataframe = make_dataframe_for_abbr(sport_df_use, 'first', 'before')

/Users/starspace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/starspace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
other_dataframe = make_dataframe_for_abbr(other_df_use, 'first', 'before')

In [8]:
link_ptrn_external = "(https?:\/\/)?(www\.)?([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d\-_\.]+)?\.[a-z]+(\/[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d_\/\?\.=\-\&\%]*)?"
link_ptrn_internal = "(file|(https?:\/\/)?localhost):(\/)*([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d_\/\?\.=\-\&\%]*)"
link_ptrn_mail = "[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d\-_\.]*@[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d\-_\.]+(\.[a-z]+)?"
link_ptrn_hashtag = "# ?[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö\d\-_]+"
link_ptrn_all = "^(" + link_ptrn_external + "|" + link_ptrn_internal + "|" + link_ptrn_mail + "|" + link_ptrn_hashtag + ")$"
ordinal_ptrn = "^(\d{1,2}(\.\d{3})*|\d+)\.$"

"""exclude web links and ordinal numbers from tokenization"""
def set_type(df):
    start = time.time()
    df.loc[(df['after'].str.match(link_ptrn_all)), 'type'] = "wlink"
    df.loc[(df['after'].str.match(ordinal_ptrn)), 'type'] = "ordinal"
    print(f"set_type done in {time.time() - start}")
    

In [9]:
set_type(sport_dataframe)

set_type done in 0.37906527519226074


In [ ]:
set_type(other_dataframe)

In [10]:
"""expands most abbreviations, a.m.k. -> að minnsta kosti"""
def replace_abbr_words(df):
    start = time.time()
    # setja allt /s, /t hérna, tonn, hektari, 
    df['after'] = df['after'].str.replace(r"^(((((\d{1,2}\.)?(\d{3}\.)*\d{3}|\d+)(,\d+)?|" + amounts + ")( )?)\s)(þús\.?)( [A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]*)?$", '\g<1>þúsund \g<11>').astype('str')
    df.loc[((df['sentbefore'].str.match("^[Mm]örg$")) & (df['after'].str.match("^þús\.?$"))), 'after'] = "þúsund"
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^gr\.$"))), 'after'] = "grein"
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^mgr\.$"))), 'after'] = "málsgrein"
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^nmgr\.$"))), 'after'] = "neðanmálsgrein"
    df['after'] = df['after'].str.replace(r"^([Kk]l ?(\.|\:)?)(\s?\d{2}[:\.]\d{2})$", 'klukkan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"(?i)^kl ?(\.|\:)? ?((núll|eitt|tvö|þrjú|fjögur|fimm|sex|sjö|átta|níu|tíu|ellefu|tólf|((þret|fjór|fimm|sex)tán)|((sau|á|ní)tján)|tuttugu( og (eitt|tvö|þrjú|fjögur))?).*)$", "klukkan \g<2>").astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]íma)?(nr\.?)$", '\g<1>númer').astype('str')
    df['after'] = df['after'].str.replace(r"([Ff]\.?[Kk]r\.?)$", ' fyrir Krist').astype('str')
    df['after'] = df['after'].str.replace(r"([Ee]\.?[Kk]r\.?)$", ' eftir Krist').astype('str')
    df['after'] = df['after'].str.replace(r"^([Cc]a|CA)\.?$", 'sirka').astype('str')
    df.loc[((df['after'].str.match("^[Ii]nnsk\.$")) & (df['sentafter'].str.match("^(blm\.?|blaðamanns)$"))), 'after'] = "innskot"
    df.loc[((df['sentbefore'].str.match("^(?i)[Ii]nnsk(\.|ot)$")) & (df['after'].str.match("^([Bb]lm|BLM)\.?$"))), 'after'] = "blaðamanns"
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^[Ss]ek\.?$"))), 'after'] = "sekúnda"
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^[Mm]ín\.?$"))), 'after'] = "mínúta"
    df['after'] = df['after'].str.replace(r"^([Nn]\.?k\.?|N\.?K\.?)$", 'næstkomandi').astype('str')
    df['after'] = df['after'].str.replace(r"^[Dd]r\.?$", 'doktor').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]ltjn|SLTJN)\.?$", 'Seltjarnarnes').astype('str')
    df['after'] = df['after'].str.replace(r"^([Rr]ví?k|RVÍ?K)$", 'Reykjavík').astype('str')
    df['after'] = df['after'].str.replace(r"^([Kk]höfn|[Kk]bh|KBH)$", 'Kaupmannahöfn').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]thlm|STHLM)$", 'Stokkhólmur').astype('str')
    df['after'] = df['after'].str.replace(r"^[Ss]\. ?s\.?$", 'svo sem').astype('str')
    df['after'] = df['after'].str.replace(r"^([Vv]s\.?|VS\.?)$", 'versus').astype('str')
    df['after'] = df['after'].str.replace(r"^([Tt]\.?d\.?|T\.?D\.?)$", 'til dæmis').astype('str')
    df['after'] = df['after'].str.replace(r"^([Aa]\.?m\.?k\.?|A\.?M\.?K\.?)$", 'að minnsta kosti').astype('str')
    df['after'] = df['after'].str.replace(r"^[Aa]\.m\.m\.?$", 'að mínu mati').astype('str')
    df['after'] = df['after'].str.replace(r"^[Aa]\.n\.l\.?$", 'að nokkru leyti').astype('str')
    df['after'] = df['after'].str.replace(r"^([Aa]lm|ALM)\.?$", 'almennt').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)ath\.?$", 'athugið').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)aths\.?$", 'athugasemd').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)atr\.?$", 'atriði').astype('str')
    df['after'] = df['after'].str.replace(r"^[Ff]él\.?$", 'félag').astype('str')
    df.loc[((df['sentbefore'].str.match("^\($")) & (df['after'].str.match("^e\.$"))), 'after'] = "enska"
    df['after'] = df['after'].str.replace(r"^([Íí]sl|ÍSL)\.?$", 'íslenska').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)e\.?h\.?f\.?$", 'E H F').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)o\.?h\.?f\.?$", 'O H F').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)h\.?f\.?$", 'H F').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)s\.?f\.?$", 'S F').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)q\.?e\.?d\.?$", 'Q E D').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ee]nnfr|ENNFR)\.?$", 'ennfremur').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ee]\.k\.?|E\.K\.?)$", 'einhvers konar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ff]rh|FRH)\.?$", 'framhald').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ff]rt|FRT)\.?$", 'framtíð').astype('str') 
    df['after'] = df['after'].str.replace(r"^[Hh]dl\.?$", 'héraðsdómslögmaður').astype('str')
    df['after'] = df['after'].str.replace(r"^[Hh]rl\.?$", 'hæstaréttarlögmaður').astype('str')
    df['after'] = df['after'].str.replace(r"^([Hh]öf|HÖF)\.?$", 'höfundur').astype('str')
    df['after'] = df['after'].str.replace(r"^[Hh]r\.?$", 'herra').astype('str')
    df['after'] = df['after'].str.replace(r"^[Hh]v\.?$", 'háttvirtur').astype('str')
    #df.loc[((df['sentbefore'].str.match("^\W$")) & (df['after'].str.match("^[Ss]r\.?$"))), 'after'] = "séra"
    #df.loc[((df['sentbefore'].str.match("^[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+$")) & (df['after'].str.match("^([Ss]r|SR)\.?$"))), 'after'] = "eldri"
    df.loc[((df['sentbefore'].str.match("^[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+$")) & (df['after'].str.match("^([Jj]r)\.?$"))), 'after'] = "yngri"
    df['after'] = df['after'].str.replace(r"^[Kk]k$", 'karlkyn').astype('str')
    df['after'] = df['after'].str.replace(r"^[Hh]v?k$", 'hvorugkyn').astype('str')
    df['after'] = df['after'].str.replace(r"^[Kk]vk$", 'kvenkyn').astype('str')
    df.loc[((df['after'].str.match("^[Kk]t\.?$")) & (df['sentafter'].str.match("^(:|\d{6}\-?\d{4})$"))), 'after'] = "kennitala"
    df.loc[((df['after'].str.match("^[Ss]\.?$")) & (df['sentafter'].str.match("^(:|\d{3}\-?\d{4})$"))), 'after'] = "sími"
    df.loc[((df['after'].str.match("^[Rr]n\.?$")) & (df['sentafter'].str.match("^(:|[\d-]+)$"))), 'after'] = "reikningsnúmer"
    df['after'] = df['after'].str.replace(r"^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)(x|\*)((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)$", '\g<1> sinnum \g<6>').astype('str')
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentafter'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['after'].str.match("^(\*|x)$"))), 'after'] = "sinnum"
    df['after'] = df['after'].str.replace(r"^[Nn]úv\.?$", 'núverandi').astype('str')
    df['after'] = df['after'].str.replace(r"^[Rr]itstj\.?$", 'ritstjóri').astype('str')
    df['after'] = df['after'].str.replace(r"^[Rr]slm\.?$", 'rannsóknarlögreglumaður').astype('str')
    df['after'] = df['after'].str.replace(r"^[Ss]br\.?$", 'samanber').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]\.?l\.?|SL\.)$", 'síðastliðinn').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]\.?k\.?)$", 'svokallað').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]kv\.?|SKV\.?)$", 'samkvæmt').astype('str')
    df['after'] = df['after'].str.replace(r"^([Tt]bl|TBL)\.?$", 'tölublað').astype('str')
    
    df['after'] = df['after'].str.replace(r"^([Tt]l\.?)$", 'tengiliður').astype('str')
    df.loc[((df['sentbefore'].str.match("^\d+\.$")) & (df['after'].str.match("^([Úú]tg|ÚTG)\.?$"))), 'after'] = "útgáfa"
    df['after'] = df['after'].str.replace(r"^([Vv]sk\.?)$", 'virðisaukaskatt').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]f\.?)$", 'þolfall').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]gf\.?)$", 'þágufall').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]lt\.?)$", 'þáliðin tíð').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ll]h\.?nt\.?)$", 'lýsingarháttur nútíðar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ll]h\.?þt\.?)$", 'lýsingarháttur þátíðar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Óó]ákv\.gr\.)$", 'óákveðinn greinir').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.a\.)$", 'meðal annars').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.?v\.?)$", 'miðað við').astype('str')
    df['after'] = df['after'].str.replace(r"^([Pp]\.?s\.?)$", 'P S').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ff]\.?hl\.?|F\.HL\.)$", 'fyrri hluti').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]\.?hl\.?|S\.HL\.)$", 'síðari hluti').astype('str')
    df['after'] = df['after'].str.replace(r"^([Tt]\.h\.)$", 'til hægri').astype('str')
    df['after'] = df['after'].str.replace(r"^([Tt]\.v\.)$", 'til vinstri').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.á\.)$", 'þessa árs').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.h\.)$", 'þess háttar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.m\.)$", 'þessa mánaðar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.e\.)$", 'það er').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ff]\.o\.t\.)$", 'fyrir okkar tímatal').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ee]\.?t\.?v\.?)$", 'ef til vill').astype('str')
    df['after'] = df['after'].str.replace(r"^([Gg]\.?r\.?f\.?|G\.?R\.?F\.?)$", 'gerum ráð fyrir').astype('str')
    df['after'] = df['after'].str.replace(r"^([Gg]\.?m\.?g\.?|G\.?M\.?G\.?)$", 'guð minn góður').astype('str')
    df['after'] = df['after'].str.replace(r"^([Hh]\.u\.b\.?|H\.U\.B\.?)$", 'hér um bil').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.a\.o\.?|M\.A\.O\.?)$", 'meðal annarra orða').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.a\.s\.?|M\.A\.S\.?)$", 'meira að segja').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.?t\.?t\.?|M\.?T\.?T\.?)$", 'með tilliti til').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]\.?ö\.?o\.?|M\.?Ö\.?O\.?)$", 'með öðrum orðum').astype('str')
    df['after'] = df['after'].str.replace(r"^([Nn]\.?t\.?t\.?|N\.?T\.T\.?)($)$", 'nánar tiltekið').astype('str')
    df['after'] = df['after'].str.replace(r"^([Oo]\.?fl\.?|O\.?FL\.?)$", 'og fleira').astype('str')
    
    df['after'] = df['after'].str.replace(r"^([Oo]\.?m\.?fl\.?|O\.?M\.?FL\.?)$", 'og margt fleira').astype('str')
    df['after'] = df['after'].str.replace(r"^([Oo]\.?s\.?frv?\.?|O\.?S\.?FRV?\.?)$", 'og svo framvegis').astype('str')
    df['after'] = df['after'].str.replace(r"^([Oo]\.?þ\.?h\.?|O\.?Þ\.?H\.?)$", 'og þess háttar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Oo]\.þ\.u\.l\.?|O\.Þ\.U\.L\.?)$", 'og því um líkt').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.?e\.?a\.?s\.?|Þ\.?E\.?A\.?S\.?)$", 'það er að segja').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.?þ\.?\.?a\.?a\.?|Þ\.?Þ\.?A\.?A\.?)$", 'þá og því aðeins að').astype('str')
    df['after'] = df['after'].str.replace(r"^([Tt]\.a\.m\.?|T\.A\.M\.)$", 'til að mynda').astype('str')
    df['after'] = df['after'].str.replace(r"^([Uu]\.?þ\.?b\.?|U\.?Þ\.?B\.?)$", 'um það bil').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.u\.l\.?|Þ\.U\.L\.)$", 'því um líkt').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.a\.l\.?|Þ\.A\.L\.)$", 'þar af leiðandi').astype('str')
    df['after'] = df['after'].str.replace(r"^([þÞ]\.á(\.| )m\.?|Þ\.Á(\.| )M\.?)$", 'þar á meðal').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.m\.t\.?|Þ\.M\.T\.?)$", 'þar með talið').astype('str')
    
    
    df['after'] = df['after'].str.replace(r"^([Mm]\.y\.s\.?)$", 'metra yfir sjávarmáli').astype('str')
    df.loc[((df['after'].str.match("^f\.?$")) & (df['sentafter'].str.match(date_ptrn))), 'after'] = "fæddur"
    df['after'] = df['after'].str.replace(r"^([Ff]\.h\.?)$", 'fyrir hönd').astype('str')
    df['after'] = df['after'].str.replace(r"^([Þþ]\.a\.?)$", 'þannig að').astype('str')
    df['after'] = df['after'].str.replace(r"^([Pp]k\.?)$", 'pakki').astype('str')
    df['after'] = df['after'].str.replace(r"^([Aa]lþm\.?)$", 'alþingismaður').astype('str')
    df['after'] = df['after'].str.replace(r"^([Áá]rg\.?)$", 'árgangur').astype('str')
    df['after'] = df['after'].str.replace(r"^([Oo](\.?|g )fl\.|O(\.?|G )FL\.)$", 'og fleira').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]amþ|SAMÞ)\.?$", 'samþykki').astype('str')
    df['after'] = df['after'].str.replace(r"^([Uu]ppl|UPPL)\.?$", 'upplýsingar').astype('str')
    df['after'] = df['after'].str.replace(r"^([Mm]fl|MFL)\.?$", 'meistaraflokkur').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ll]td|LTD)\.?$", 'limited').astype('str')
    df['after'] = df['after'].str.replace(r"^([Uu]td|UTD)\.?$", 'united').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]gt\.?)$", 'sergeant').astype('str')
    df['after'] = df['after'].str.replace(r"^([Ss]t\.?)$", 'saint').astype('str')
    df['after'] = df['after'].str.replace(r"^([Nn]kl|NKL)$", 'nákvæmlega').astype('str')
    df.loc[((df['after'].str.match("^frák\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\/\d{1,2}$"))), 'after'] = "fráköst"
    df['after'] = df['after'].str.replace(r"^(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")( )?)(m\^2)$", '\g<1> fermetrar').astype('str')
    df['after'] = df['after'].str.replace(r"^(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")( )?)(m\^3)$", '\g<1> rúmmetrar').astype('str')
    df['after'] = df['after'].str.replace(r"^\+([1-9])", 'plús \g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^\-([1-9])", 'mínus \g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(bls\.?)(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")( )?)$", 'blaðsíða \g<1>').astype('str')
    df.loc[((df['after'].str.match("^[Bb]ls\.?$")) & (df['sentafter'].str.match("^(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")( )?)$"))), 'after'] = "blaðsíða"
    # degrees
    df['after'] = df['after'].str.replace(r"([02-9])( )?°C$", '\g<1> gráður selsíus').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°C$", '\g<1> gráða selsíus').astype('str')
    df['after'] = df['after'].str.replace(r"([02-9])( )?°F$", '\g<1> gráður fahrenheit').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°F$", '\g<1> gráða fahrenheit').astype('str')
    # direction degrees
    df['after'] = df['after'].str.replace(r"([02-9])( )?°W$", '\g<1> gráður vestur').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°W$", '\g<1> gráða vestur').astype('str')
    df['after'] = df['after'].str.replace(r"([02-9])( )?°N$", '\g<1> gráður norður').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°N$", '\g<1> gráða norður').astype('str')
    df['after'] = df['after'].str.replace(r"([02-9])( )?°E$", '\g<1> gráður austur').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°E$", '\g<1> gráða austur').astype('str')
    df['after'] = df['after'].str.replace(r"([02-9])( )?°S$", '\g<1> gráður suður').astype('str')
    df['after'] = df['after'].str.replace(r"(1)( )?°S$", '\g<1> gráða suður').astype('str')
    print(f"abbreviations done in {time.time() - start}")


In [11]:
"""replaces an abbreviation after a slash, m/s -> m á sekúndu"""
def denominator_measurement(df):
    start = time.time()
    df['after'] = df['after'].str.replace(r"\/t\.?$", ' á tonnið').astype('str')
    df['after'] = df['after'].str.replace(r"\/ha\.?$", ' á hektarann').astype('str')
    df['after'] = df['after'].str.replace(r"\/ng\.?$", ' á nanógrammið').astype('str')
    df['after'] = df['after'].str.replace(r"\/mg\.?$", ' á milligrammið').astype('str')
    df['after'] = df['after'].str.replace(r"\/µg\.?$", ' á míkrógrammið').astype('str')
    df['after'] = df['after'].str.replace(r"\/gr?\.?$", ' á grammið').astype('str')
    df['after'] = df['after'].str.replace(r"\/kg?$", ' á kílóið').astype('str')
    df['after'] = df['after'].str.replace(r"\/lbs$", ' á pundið').astype('str')
    df['after'] = df['after'].str.replace(r"\/ml$", ' á millilítrann').astype('str')
    df['after'] = df['after'].str.replace(r"\/dl$", ' á desilítrann').astype('str')
    df['after'] = df['after'].str.replace(r"\/[sc]l$", ' á sentilítrann').astype('str')
    df['after'] = df['after'].str.replace(r"\/l$", ' á lítrann').astype('str')
    df['after'] = df['after'].str.replace(r"\/tsk$", ' á teskeið').astype('str')
    df['after'] = df['after'].str.replace(r"\/msk$", ' á matskeið').astype('str')
    df['after'] = df['after'].str.replace(r"\/ft$", ' á fetið').astype('str')
    df['after'] = df['after'].str.replace(r"\/m\.?$", ' á metrann').astype('str')
    df['after'] = df['after'].str.replace(r"\/pm\.?$", ' á píkómetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/nm\.?$", ' á nanómetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/[cs]m\.?$", ' á sentimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/dm\.?$", ' á desimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/km\.?$", ' á kílómetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/Nm\.?$", ' á Njútonmetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/klst$", ' á klukkustund').astype('str')
    df['after'] = df['after'].str.replace(r"\/kw\.?(st|h)\.?$", ' á kílóvattstund').astype('str')
    df['after'] = df['after'].str.replace(r"\/Mw\.?(st|h)\.?$", ' á megavattstund').astype('str')
    df['after'] = df['after'].str.replace(r"\/Gw\.?(st|h)\.?$", ' á gígavattstund').astype('str')
    df['after'] = df['after'].str.replace(r"\/Tw\.?(st|h)\.?$", ' á teravattstund').astype('str')
    df['after'] = df['after'].str.replace(r"\/s(ek)?\.?$", ' á sekúndu').astype('str')
    df['after'] = df['after'].str.replace(r"\/mín\.?$", ' á mínútu').astype('str')
    df['after'] = df['after'].str.replace(r"\/ms(ek)?\.?$", ' á millisekúndu').astype('str')
    df['after'] = df['after'].str.replace(r"\/kr\.?$", ' á krónu').astype('str')
    df['after'] = df['after'].str.replace(r"\/fm\.?$", ' á fermetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/ferm\.?$", ' á fermetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/m²$", ' á fermetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/m2$", ' á fermetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/rúmm\.?$", ' á rúmmetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/m³$", ' á rúmmetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/m3$", ' á rúmmetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/mm²$", ' á fermillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/mm2$", ' á fermillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/mm³$", ' á rúmmillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/mm3$", ' á rúmmillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/[cs]m²$", ' á fermillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/[cs]m2$", ' á fermillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/[cs]m³$", ' á rúmmillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/[cs]m3$", ' á rúmmillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/km²$", ' á ferkílómillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/km2$", ' á ferkílómillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/km³$", ' á rúmkílómillimetra').astype('str')
    df['after'] = df['after'].str.replace(r"\/km3$", ' á rúmkílómillimetra').astype('str')
    
    df['after'] = df['after'].str.replace(r"\/\%$", ' á prósentið').astype('str')
    df['after'] = df['after'].str.replace(r"\/stk\.?$", ' á stykkið').astype('str')
    df['after'] = df['after'].str.replace(r"\/V$", ' á volt').astype('str')
    df['after'] = df['after'].str.replace(r"\/kV$", ' á kílóvolt').astype('str')
    df['after'] = df['after'].str.replace(r"\/Hz", ' á herz').astype('str')
    df['after'] = df['after'].str.replace(r"\/kHz$", ' á kílóherz').astype('str')
    df['after'] = df['after'].str.replace(r"\/MHz$", ' á megaherz').astype('str')
    df['after'] = df['after'].str.replace(r"\/GHz$", ' á gígaherz').astype('str')
    df['after'] = df['after'].str.replace(r"\/W$", ' á vatt').astype('str')
    df['after'] = df['after'].str.replace(r"\/kW$", ' á kílóvatt').astype('str')
    print(f"demoniator_measure done in {time.time() - start}")

In [12]:
"""expands abbreviations for directions, SV-átt -> suðvestanátt"""
def directions(df):
    start = time.time()
    df.loc[((df['after'].str.match("^SV$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "suðvestur"
    df.loc[((df['after'].str.match("^SSV$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "suðsuðvestur"
    #df.loc[((df['after'].str.match("^V$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "vestur"
    df.loc[((df['after'].str.match("^NV$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "norðvestur"
    df.loc[((df['after'].str.match("^NNV$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "norðnorðvestur"
    #df.loc[((df['after'].str.match("^N$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "norður"
    df.loc[((df['after'].str.match("^NA$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "norðaustur"
    df.loc[((df['after'].str.match("^NNA$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "norðaustur"
    #df.loc[((df['after'].str.match("^A$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "austur"
    df.loc[((df['after'].str.match("^SA$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "suðaustur"
    df.loc[((df['after'].str.match("^SSA$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "suðsuðaustur"
    #df.loc[((df['after'].str.match("^S$")) & (df['sentafter'].str.match("^(\-|–|—)?$"))), 'after'] = "suður"

    df['after'] = df['after'].str.replace(r"^(SV(\-|–|—))(?!(til|lands|átt|verðu|vert))([A-ZÁÐÉÍÓÚÝÞÆÖ][A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+)$", 'suðvestur\g<2>\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NV(\-|–|—))(?!(til|lands|átt|verðu|vert))([A-ZÁÐÉÍÓÚÝÞÆÖ][A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+)$", 'norðvestur\g<2>\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NA(\-|–|—))(?!(til|lands|átt|verðu|vert))([A-ZÁÐÉÍÓÚÝÞÆÖ][A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+)$", 'norðaustur\g<2>\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(SA(\-|–|—))(?!(til|lands|átt|verðu|vert))([A-ZÁÐÉÍÓÚÝÞÆÖ][A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+)$", 'suðaustur\g<2>\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(A(\-|–|—))([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]*sýsl[ua])$", 'austur\g<2>\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(S(\-|–|—))([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]*sýsl[ua])$", 'suður\g<2>\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(V(\-|–|—))([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]*sýsl[ua])$", 'vestur\g<2>\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(N(\-|–|—))([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]*sýsl[ua])$", 'norður\g<2>\g<3>').astype('str')

    df['after'] = df['after'].str.replace(r"^(SV(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'suðvestur\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(V(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'vestur\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NV(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'norðvestur\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(N(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'norður\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NA(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'norðaustur\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(SA(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'suðaustur\g<4>').astype('str')
    df['after'] = df['after'].str.replace(r"^(S(\-|–|—))(?!(til|lands|átt|verðu|vert))([a-záðéíóúýþæö]+)$", 'suður\g<4>').astype('str')

    df['after'] = df['after'].str.replace(r"^(SV(\-|–|—))(lands|átt|verðu|vert)$", 'suðvestan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(V(\-|–|—))(lands|átt|verðu|vert)$", 'vestan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NV(\-|–|—))(lands|átt|verðu|vert)$", 'norðvestan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(N(\-|–|—))(lands|átt|verðu|vert)$", 'norðan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NA(\-|–|—))(lands|átt|verðu|vert)$", 'norðaustan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(A(\-|–|—))(lands|átt|verðu|vert)$", 'austan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(SA(\-|–|—))(lands|átt|verðu|vert)$", 'suðaustan\g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(S(\-|–|—))(lands|átt|verðu|vert)$", 'sunnan\g<3>').astype('str')

    df['after'] = df['after'].str.replace(r"^(A(\-|–|—))(land|landi)$", 'austur\g<3>').astype('str')

    df['after'] = df['after'].str.replace(r"^(SV(\-|–|—))(til)$", 'suðvestan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(V(\-|–|—))(til)$", 'vestan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NV(\-|–|—))(til)$", 'norðvestan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(N(\-|–|—))(til)$", 'norðan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(NA(\-|–|—))(til)$", 'norðaustan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(A(\-|–|—))(til)$", 'austan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(SA(\-|–|—))(til)$", 'suðaustan \g<3>').astype('str')
    df['after'] = df['after'].str.replace(r"^(S(\-|–|—))(til)$", 'sunnan \g<3>').astype('str')
    print(f"directions done in {time.time() - start}")

In [13]:
# kemur bara ef það er núll á undan

wlink_numbers = [('0', 'núll'),
                 ('1', 'einn'),
                 ('2', 'tveir'),
                 ('3', 'þrír'),
                 ('4', 'fjórir'),
                 ('5', 'fimm'),
                 ('6', 'sex'),
                 ('7', 'sjö'),
                 ('8', 'átta'),
                 ('9', 'níu'),
                 ('\.', 'punktur'),
                 ('\-', 'bandstrik'),
                 ('\/', 'skástrik'),
                 ('_', 'undirstrik'),
                 ('@', 'hjá'),
                 (':', 'tvípunktur'),
                 ('=', 'jafnt og'),
                 ('\?', 'spurningarmerki'),
                 ('!', 'upphrópunarmerki'),
                 ('&', 'og'),
                 ('%', 'prósent'),
                 ('#', 'myllumerki')]


def wlink_fun(substr):
    substr = " ".join(substr)
    for symbol, word in wlink_numbers:
        substr = re.sub(symbol, word, substr)
    return substr

"""expands weblinks, www.mbl.is/innlent -> w w w punktur m b l punktur i s skástrik i n n l e n t"""
def replace_weblinks(df):
    start = time.time()
    for i in range(len(df)):
        if df['type'].iloc[i] == 'wlink':
            df['after'].iloc[i] = wlink_fun(df['after'].iloc[i])
        else:
            pass
    print(f"replace_weblinks done in {time.time() - start}")

In [14]:
"""separate words that are like 3N4L to 3 N 4 L, also from % and °"""
def separate(df):
    start = time.time()
    df['after'] = df['after'].str.replace(r"^(?i)2ja$", 'tveggja').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)3ja$", 'þriggja').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)4ð(a|i)$", 'fjórð\g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)5t(a|i)$", 'fimmt\g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)6t(a|i)$", 'sjött\g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)7d(a|i)$", 'sjöund\g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)8d(a|i)$", 'áttund\g<1>').astype('str')
    df['after'] = df['after'].str.replace(r"^(?i)9d(a|i)$", 'níund\g<1>').astype('str')
    
    df['after'] = df['after'].str.replace(r"(?i)([a-záðéíóúýþæö]+)(\d+)", '\g<1> \g<2>').astype('str')
    df['after'] = df['after'].str.replace(r"(?i)(\d+)([a-záðéíóúýþæö]+)", '\g<1> \g<2>').astype('str')
    df['after'] = df['after'].str.replace(r"^([A-ZÁÐÉÍÓÚÝÞÆÖ]+)(\-[A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+)$", "\g<1> \g<2>").astype("str")
    df['after'] = df['after'].str.replace(r"^([A-ZÁÐÉÍÓÚÝÞÆÖa-záðéíóúýþæö]+\-)([A-ZÁÐÉÍÓÚÝÞÆÖ]+)$", "\g<1> \g<2>").astype("str")
    df['after'] = df['after'].str.replace(r"(?i)([\da-záðéíóúýþæö]+)(°)", '\g<1> \g<2>').astype('str')
    df['after'] = df['after'].str.replace(r"(?i)([\da-záðéíóúýþæö]+)(\%)", '\g<1> \g<2>').astype('str')
    print(f"separate done in {time.time() - start}")

In [15]:
"""separate 02.12.2014 to 02. 12. 2014 so they’re read as two ordinals and one cardinal"""
def dates_in_numbers(df):
    start = time.time()
    df['after'] = df['after'].str.replace(r"^(0?[1-9]|[12]\d|3[01])\.(0?[1-9]|1[012])\.(\d{3,4})$", " \g<1>. \g<2>. \g<3>").astype('str')
    df['after'] = df['after'].str.replace(r"^(0?[1-9]|[12]\d|3[01])\.(0?[1-9]|1[012])\.$", " \g<1>. \g<2>.").astype('str')
    print(f"dates_in_numbers done in {time.time() - start}")

In [16]:
"""561 1234 -> 561-1234 so it won’t be tokenized separately"""
def phone_numbers(df):
    start = time.time()
    df['after'] = df['after'].str.replace("(\d{3})( )(\d{4})", " \g<1>-\g<3>").astype('str')
    print(f"phone_numbers done in {time.time() - start}")

In [17]:
def first_functions(df):
    replace_abbr_words(df)
    denominator_measurement(df)
    directions(df)
    replace_weblinks(df)

In [18]:
first_functions(sport_dataframe)

abbreviations done in 5.51167106628418
demoniator_measure done in 0.5761139392852783
directions done in 2.506467819213867
replace_weblinks done in 0.1289970874786377


/Users/starspace/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
first_functions(other_dataframe)

In [19]:
"""compares whether a word was changed, marking it as an abbreviation, not necessary in tts"""
def df_comparison(df, col1, col2, typecol):
    start = time.time()
    df['compare'] = df[col1] == df[col2]
    df.loc[((df['compare'] == False) & (df['type'] == "")), 'type'] = typecol
    print(f"df_comparison finished in {time.time() - start}")

In [20]:
df_comparison(sport_dataframe, 'before', 'after', 'abbr')

df_comparison finished in 0.20363473892211914


In [ ]:
df_comparison(other_dataframe, 'before', 'after', 'abbr')

In [21]:
def second_functions(df):
    separate(df)
    dates_in_numbers(df)
    phone_numbers(df)

In [22]:
second_functions(sport_dataframe)

separate done in 0.39779210090637207
dates_in_numbers done in 0.05438089370727539
phone_numbers done in 0.024616241455078125


In [ ]:
second_functions(other_dataframe)

## MAKE A FLAT DATAFRAME FOR MEASURE

In [23]:
"""tokenize the words after an expansion of abbreviations, skip weblinks and ordinals (they are split up incorrectly)"""
def tokenize_nsw(df, after_column):
    start = time.time()
    print(f"length of dataframe is {len(df)}")
    # tokenize every cell of the dataframe to keep structure
    for i in tqdm(range(len(df))):
        if df['type'].iloc[i] == 'wlink' or df['type'].iloc[i] == 'ordinal':
            df[after_column].iloc[i] = [df[after_column].iloc[i]]
        else:
            df[after_column].iloc[i] = [sentence.split() for sentence in split_into_sentences(df[after_column].iloc[i])][0]
    print(f"tokenize_nsw is done in {time.time() - start}")

"""remove words from brackets"""
def flatten_nsw(df):
    start = time.time()
    # split expanded words into one word a row
    df_expanded = pd.DataFrame([
        [s_nr, w_nr, before, a, tag] for s_nr, w_nr, before, after, tag in df.values for a in after
    ], columns=df.columns)
    print(f"flatten_nsw is done in {time.time() - start}")
    
    return df_expanded

In [24]:
tokenize_nsw(sport_dataframe, 'after')

length of dataframe is 12106



tokenize_nsw is done in 4.458677053451538


In [ ]:
tokenize_nsw(other_dataframe, 'after')

In [25]:
sport_expanded = flatten_nsw(sport_dataframe[['sentence_id', 'token_id', 'before', 'after', 'type']])

flatten_nsw is done in 0.030373811721801758


In [ ]:
other_expanded = flatten_nsw(other_dataframe[['sentence_id', 'token_id', 'before', 'after', 'type']])

In [26]:
sport_dataframe_expanded = make_dataframe_for_abbr(sport_expanded, 'second', 'after')

In [ ]:
other_dataframe_expanded = make_dataframe_for_abbr(other_expanded, 'second', 'after')

In [27]:
sport_dataframe_expanded['after_measure'] = sport_dataframe_expanded['after']
set_type(sport_dataframe_expanded)

set_type done in 0.4164271354675293


In [ ]:
other_dataframe_expanded['after_measure'] = other_dataframe_expanded['after']
set_type(other_dataframe_expanded)

## MEASUREMENTS

#### Tonn

In [28]:
def df_tonn(df):
    print("df_tonn")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonn"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonn"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*1|\d+(,\d*)1))$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonni"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonns"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonnum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonnum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonna"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonna"       
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonn"
    df.loc[((df['sent2before'].str.match("(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^t\.?$"))), 'after_measure'] = "tonn"

#### Hektarar

In [29]:
def df_hektari(df):
    print("df_hektari")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektara"
    df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektara"
    df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektara"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hekturum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hekturum"  
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektari" 
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektarar"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^ha\.?$"))), 'after_measure'] = "hektarar"    


#### somegram

In [30]:
prefix_weight = [('nanó', 'n'),
                 ('milli', 'm'),
                 ('míkró', 'µ'),
                 ('píkó', 'p'),
                 ('attó', 'a'),
                 ('zeptó', 'z'),
                 ('yoktó', 'y')]

def df_somegram(df):
    print("df_somegram")
    for prefix, letter in tqdm(prefix_weight):
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "gramm"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grammi"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "gramms"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömm"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömm"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömmum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömmum"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "gramma"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "gramma"       
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "gramm" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömm"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "g\.?$"))), 'after_measure'] = prefix + "grömm"  

    
def df_somegram_noprior(df):
    print("df_somegram_noprior")
    for prefix, letter in tqdm(prefix_weight[:3]):
        df.loc[(df['after_measure'].str.match("^" + letter + "g\.?$")), 'after_measure'] = prefix + "grömm"  


#### gram

In [31]:
def df_gram(df):
    print("df_gram")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "gramm"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grammi"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "gramms"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömm"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömm"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömmum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömmum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "gramma"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "gramma"       
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "gramm" 
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömm"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^gr?\.?$"))), 'after_measure'] = "grömm"  


#### basegram

In [32]:
base_weight = [('kíló', 'kg'),
               ('pund', 'lbs')]

def df_basegram(df):
    print("df_basegram")
    for word, letters in tqdm(base_weight):
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "i"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "s"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "um"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "um"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "a"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word + "a"       
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = word 
    
        df.loc[(df['after_measure'].str.match("^" + letters + "\.?$")), 'after_measure'] = word 


#### someliter

In [33]:
prefix_liter = [('', ''),
                 ('d', 'desi'),
                 ('c', 'senti'),
                 ('m', 'milli'),
                 ('µ', 'míkró')]


def df_someliter(df):
    print("df_someliter")
    for letter, prefix in tqdm(prefix_liter):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítra"
        df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítra"
        df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9])?)|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítra"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9])?)|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítrum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítrum"  
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítri" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9](,\d+)?)|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítrar"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ll]\.?$"))), 'after_measure'] = prefix + "lítrar"

    
def df_someliter_no_prior(df):
    print("df_someliter_noprior")
    for letter, prefix in tqdm(prefix_liter[1:]):
        df.loc[((df['after_measure'].str.match("^" + letter + "l\.?$"))), 'after_measure'] = prefix + "lítrar"   


#### somespoon

In [34]:
prefix_spoon = [('t', 'te'),
                ('m', 'mat')]
def df_somespoon(df):
    print("df_somespoon")
    for letter, prefix in tqdm(prefix_spoon):
        df.loc[((df['sent2before'].str.match("^"+ accdat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeið"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðar"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðar"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðar"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðum"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiða"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiða"       
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeið"
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðar"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "sk\.?$"))), 'after_measure'] = prefix + "skeiðar"

        df.loc[(df['after_measure'].str.match("^" + letter + "sk\.?$")), 'after_measure'] = prefix + "skeiðar"


#### Fet


In [35]:
def df_fet(df):
    print("df_fet")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fet"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fet"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "feti"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fets"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fetum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fetum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "feta"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "feta"       
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fet"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^(\′|ft\.?)$"))), 'after_measure'] = "fet"


#### tomma

In [36]:
def df_inch(df):
    print("df_inch")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tommu"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tommur"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tommum"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tomma"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tomma"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^\″$"))), 'after_measure'] = "tommur" 

    

#### metri

In [37]:
def df_metri(df):
    print("df_metri")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metra"
    df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metra"
    df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metra"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metrum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metrum"  
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metri" 
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metrar"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m\.?$")) & (df['sentafter'].str.match(r"^(?![kmgyabefstvö]\.)(.*)$"))), 'after_measure'] = "metrar"    


#### somemeter

In [38]:
prefix_meter = [('p', 'píkó'),
                ('n', 'nanó'),
                ('µ', 'míkró'),
                ('m', 'milli'),
                ('[cs]', 'senti'),
                ('d', 'desi'),
                ('k', 'kíló'),
                ('N', 'njúton')]

def df_somemeter(df):
    print("df_somemeter")
    for letter, prefix in tqdm(prefix_meter):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metrum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metrum"  
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metri" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metrar"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m\.?$"))), 'after_measure'] = prefix + "metrar"    
        
        df.loc[(df['after_measure'].str.match("^" + letter + "m\.?$")), 'after_measure'] = prefix + "metrar"    


#### kwh

In [39]:
watt_prefix = [('', ''), ('k', 'kíló'), ('M', 'Mega'), ('G', 'Gíga'), ('T', 'Tera')]

def df_kwst(df):
    print("df_kwst")
    for letter, prefix in tqdm(watt_prefix):
        df.loc[((df['sent2before'].str.match("^"+ accdat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstund"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundar"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundir"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundir"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundum"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstunda"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstunda"       
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstund"
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundir"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$"))), 'after_measure'] = prefix + "vattstundir"

        df.loc[(df['after_measure'].str.match("^" + letter + "[Ww]\.?(st|h)\.?$")), 'after_measure'] = prefix + "vattstundir"


#### klst

In [40]:
def df_klst(df):
    print("df_klst")
    df.loc[((df['sent2before'].str.match("^"+ accdat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustund"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundar"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundir"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundir"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustunda"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustunda"       
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustund"
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundir"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^klst\.?$"))), 'after_measure'] = "klukkustundir"

    df.loc[(df['after_measure'].str.match("^klst\.?$")), 'after_measure'] = "klukkustundir"


#### sometime

In [41]:
prefix_time = [('mín()?', 'mínút'),
               ('s(ek)?', 'sekúnd'),
               ('ms(ek)?', 'millisekúnd')]

def df_sometime(df):
    print("df_sometime")
    for letters, prefix in tqdm(prefix_time):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "u"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "ur"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "ur"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "um"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "um"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "na"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "na"
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "a" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "ur"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "\.?$"))), 'after_measure'] = prefix + "ur"



#### króna

In [42]:
def df_crown(df):
    print("df_crown")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónu"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónur"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónur"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónum"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "króna"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "króna"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "króna"
    df.loc[((df['sentafter'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "króna" 
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónur"
    df.loc[((df['sentafter'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónur"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^kr\.?\-?$"))), 'after_measure'] = "krónur"    

    df['after_measure'] = df['after_measure'].str.replace(r"^kr\. (\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$", '\g<1> króna').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^kr\.(\d{1,2}\.)?(\d{3}\.?)*\d*([02-9]|(,\d*)[02-9])$", '\g<1> krónur').astype('str')


#### currencies

In [43]:
def df_dollari(df):
    print("df_dollari")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollara"
    df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollara"
    df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollara"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollurum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollurum"  
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollari" 
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollarar"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\$$"))), 'after_measure'] = "dollarar"
        
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)\$$", '\g<1> dollari').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d*([02-9]|(,\d*)[02-9])\$$", '\g<1> dollarar').astype('str')
    
    df['after_measure'] = df['after_measure'].str.replace(r"^\$((\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1))$", '\g<1> dollari').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^\$((\d{1,2}\.)?(\d{3}\.?)*\d*([02-9]|(,\d*)[02-9]))$", '\g<1> dollarar').astype('str')
    
    df['after_measure'] = df['after_measure'].str.replace(r"^\$$", 'dollarar').astype('str')


In [44]:
def df_pound(df):
    print("df_pound")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pund"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pund"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pundi"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "punds"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pundum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pundum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "punda"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "punda"       
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pund"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^£$"))), 'after_measure'] = "pund"

    df['after_measure'] = df['after_measure'].str.replace(r"^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)£$", '\g<1> pund').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^£((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)$", '\g<1> pund').astype('str')
    
    df['after_measure'] = df['after_measure'].str.replace(r"^£$", 'pund').astype('str')


In [45]:
def df_yen(df):
    print("df_yen")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japanskt jen"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönsku jeni"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japansks jens"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönsk jen"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönsk jen"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönskum jenum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönskum jenum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japanskra jena"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japanskra jena"       
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japanskt jen"
    df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönsk jen"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^¥$"))), 'after_measure'] = "japönsk jen"
    
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)¥$", '\g<1> japanskt jen').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))¥$", '\g<1> japönsk jen').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)¥$", '\g<1> japanskt jen').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))¥$", '\g<1> japönsk jen').astype('str')
    
    df['after_measure'] = df['after_measure'].str.replace(r"^¥$", 'japönsk jen').astype('str')


In [46]:
currency_list = [('evr', '€'), ('rúpí', '₹'), ('lír', '₤')]

def df_currencies(df):
    print("df_currencies")
    for word, symbol in tqdm(currency_list):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "u"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "ur"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "ur"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "um"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "um"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "a"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "a"
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "a"
        df.loc[((df['sentafter'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "a" 
        df.loc[((df['sentbefore'].str.match("^(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)|" + amounts + ")$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "ur"
        df.loc[((df['sentafter'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "ur"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + symbol + "$"))), 'after_measure'] = word + "ur"
        
        df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)" + symbol + "$", '\g<1> ' + word + 'a').astype('str')
        df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))" + symbol + "$", '\g<1> ' + word + 'ur').astype('str')
        df['after_measure'] = df['after_measure'].str.replace(r"^" + symbol + "(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$", '\g<1> ' + word + 'a').astype('str')
        df['after_measure'] = df['after_measure'].str.replace(r"^" + symbol + "(\d{1,2}\.)?(\d{3}\.?)*([02-9]|\d+(,\d*[02-9]))$", '\g<1> ' + word + 'ur').astype('str')
        
        df['after_measure'] = df['after_measure'].str.replace(r"^" + symbol + "$", word + 'ur').astype('str')


#### Currencies in letters

In [47]:
currency_letters = [('ISK', 'íslenskar krónur'), ('GBP', 'sterlingspund'), ('EUR', 'evrur'),
                    ('USD', 'bandaríkjadalir'), ('DKK', 'danskar krónur'), ('AUD', 'ástralskir dalir'),
                    ('JPY', 'japönsk jen'), ('CHF', 'svissneskir frankar'), ('CAD', 'kanadískir dalir'),
                    ('CZK', 'tékkneskar krónur'), ('INR', 'indverskar rúpíur'), ('SEK', 'sænskar krónur'),
                    ('NOK', 'norskar krónur'), ('PTE', 'portúgalskir skútar')]

def df_cur_letters(df):
    print("df_cur_letters")
    for letters, word in tqdm(currency_letters):
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = word
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = word
        df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?" + letters + "$", '\g<1> ' + word).astype('str')
        df['after_measure'] = df['after_measure'].str.replace(r"^" + letters + "(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$", '\g<1> ' + word).astype('str')


#### thousand krona

In [48]:
def df_thousand_krona(df):
    print("df_thousand_krona")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['after_measure'].str.match("^þ\.?kr\.?\-?$"))), 'after_measure'] = "þúsund krónur"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['after_measure'].str.match("^þ\.?kr\.?\-?$"))), 'after_measure'] = "þúsund krónum"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['after_measure'].str.match("^þ\.?kr\.?\-?$"))), 'after_measure'] = "þúsund króna"
    df.loc[(df['after_measure'].str.match("^þ\.?kr\.?\-?$")), 'after_measure'] = "þúsund krónur"


#### million krona

In [49]:
million_list = [('m\.?kr\.?\-?', ' króna'),
                ('mljó\.?', '')]

def df_million_krona(df):
    print("df_million_krona")
    for letters, suffix in tqdm(million_list):
        df.loc[((df['sent2before'].str.match("^"+ accdat_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljón" + suffix
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljónar" + suffix
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljónir" + suffix
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljónum" + suffix
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljóna" + suffix
        
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljón" + suffix
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljónir" + suffix


#### billion krona

In [50]:
billion_list = [('ma\.?kr\.?\-?', ' króna'),
                ('mlja\.?', '')]

def df_billion_krona(df):
    print("df_billion_krona")
    for letters, suffix in tqdm(billion_list):
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarð" + suffix
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarði" + suffix
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarðs" + suffix
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarða" + suffix
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljörðum" + suffix
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarða" + suffix
        
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarður" + suffix
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))$")) & (df['after_measure'].str.match("^" + letters + "$"))), 'after_measure'] = "milljarðar" + suffix


#### dimensionmeter

In [51]:
dimension_after = [('²', 'fer'),
                   ('2', 'fer'),
                   ('³', 'rúm'),
                   ('3', 'rúm')]


def df_dimensionmeter_after(df):
    print("df_dimensionmeter_after")
    for superscript, prefix in tqdm(dimension_after):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^((\d{3}\.?)*\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{3}\.?)*\d*[02-9]|\d+(,\d*[02-9])|" + amounts + ")$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{3}\.?)*\d*[02-9]|\d+(,\d*[02-9])|" + amounts + ")$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metrum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metrum"  
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metri" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metrar"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^m" + superscript + "$"))), 'after_measure'] = prefix + "metrar"    
        
        df.loc[(df['after_measure'].str.match("^m" + superscript + "$")), 'after_measure'] = prefix + "metrar"


In [52]:

dimension_before = [('f', 'fer'),
                    ('fer', 'fer'),
                    ('rúm', 'rúm')]

def df_dimensionmeter_before(df):
    print("df_dimensionmeter_before")
    for letters, prefix in tqdm(dimension_before):
        df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metra"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metrum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metrum"  
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metri" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metrar"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letters + "m\.?$"))), 'after_measure'] = prefix + "metrar"    
        
        df.loc[(df['after_measure'].str.match("^" + letters + "m\.?$")), 'after_measure'] = prefix + "metrar" 


In [53]:
prefix_meter_dimension = [('p', 'píkó'),
                ('n', 'nanó'),
                ('µ', 'míkró'),
                ('m', 'milli'),
                ('[cs]', 'senti'),
                ('d', 'desi'),
                ('k', 'kíló')]


def df_prefix_dimensionmeter_after(df):
    print("df_prefix_dimensionmeter_after")
    for letter, prefix in tqdm(prefix_meter_dimension):
        for superscript, string in dimension_after:
            df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metrum"
            df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metrum"  
            df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metri" 
            df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metrar"
            df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + "m" + superscript + "$"))), 'after_measure'] = string + prefix + "metrar"    

            df.loc[(df['after_measure'].str.match("^" + letter + "m" + superscript + "$")), 'after_measure'] = string + prefix + "metrar"



In [54]:
def df_prefix_dimensionmeter_before(df):
    print("df_prefix_dimensionmeter_before")
    for letter, prefix in tqdm(prefix_meter_dimension):
        for preprefix, string in dimension_before:
            df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent3before'].str.match("^"+ accgen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent2before'].str.match("^"+ accgen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metra"
            df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix +"metrum"
            df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix +"metrum"  
            df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metri" 
            df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metrar"
            df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + preprefix + letter + "m\.?$"))), 'after_measure'] = string + prefix + "metrar"    

            df.loc[(df['after_measure'].str.match("^" + preprefix + letter + "m\.?$")), 'after_measure'] = string + prefix + "metrar" 


#### percent

In [55]:
def df_percent(df):
    print("df_percent")
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósent"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósent"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósenti"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósents"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósentum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósentum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósenta"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósenta"       
    df.loc[((df['sentbefore'].str.match("^(((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?)|" + amounts + ")$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósent"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?)$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósent"

    df.loc[((df['sentbefore'].str.match("^"+ acc_words + "$")) & (df['after_measure'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")\%$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósent"
    df.loc[((df['sentbefore'].str.match("^"+ dat_words + "$")) & (df['after_measure'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))\%$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósenti"
    df.loc[((df['sentbefore'].str.match("^"+ gen_words + "$")) & (df['after_measure'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))\%$")) & (df['after_measure'].str.match("^\%$"))), 'after_measure'] = "prósents"
    df.loc[((df['sentbefore'].str.match("^"+ dat_words + "$")) & (df['after_measure'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")\%$"))), 'after_measure'] = "prósentum"  
    df.loc[((df['sentbefore'].str.match("^"+ gen_words + "$")) & (df['after_measure'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")\%$"))), 'after_measure'] = "prósenta"       
    
    df['after_measure'] = df['after_measure'].str.replace(r"^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?\%$", '\g<1> prósent').astype('str')
    df['after_measure'] = df['after_measure'].str.replace(r"^\%$", 'prósent').astype('str')


#### stykki

In [56]:
def df_stykki(df):
    print("df_stykki")
    df.loc[((df['sent2before'].str.match("^"+ accdat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykki"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?)$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykki"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykkis"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykkjum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?)$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykkjum"    
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykkja"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?)$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykkja"       
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykki"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?)$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^stk\.?$"))), 'after_measure'] = "stykki"


#### electronics

In [57]:
measurement = [('V', 'volt'), ('Hz', 'herz')]

def df_electronic(df):
    print("df_electronic")
    for letter, prefix in tqdm(watt_prefix):
        for symbol, word in measurement: 
            df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word
            df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word 
            df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*1|\d+(,\d*)1))$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "i"
            df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "s"
            df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "um"
            df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "um"    
            df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "a"
            df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word + "a"       
            df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word
            df.loc[((df['sent2before'].str.match("(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^" + letter + symbol + "$"))), 'after_measure'] = prefix + word


In [58]:
def df_watt(df):
    print("df_watt")
    for letter, prefix in tqdm(watt_prefix):
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatt"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatti"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatts"
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vött"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vött"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vöttum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vöttum"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatta"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatta"       
        df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vatt" 
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vött"
        df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("(?i)^" + letter + "W\.?$"))), 'after_measure'] = prefix + "vött"  


In [59]:
byte_prefix = [('[Kk]', 'kíló'), ('M', 'mega'), ('G', 'gíga'), ('T', 'tera')]

def df_byte(df):
    print("df_byte")
    for letter, prefix in tqdm(byte_prefix):
        df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæt"
        df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæt" 
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*1|\d+(,\d*)1))$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæti"
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1))$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæts"
        df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bætum"
        df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bætum"    
        df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæta"
        df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæta"       
        df.loc[((df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæt"
        df.loc[((df['sent2before'].str.match("(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^"+ letter + "B$"))), 'after_measure'] = prefix + "bæt"


#### calorie

In [60]:
def df_cal(df):
    print("df_cal")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríu"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríur"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríur"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríum"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloría"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloría"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloría"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríur"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^[Kk][Cc]al$"))), 'after_measure'] = "kílókaloríur"    


#### degree

In [61]:
def df_degree(df):
    print("df_degree")
    df.loc[((df['sent2before'].str.match("^"+ accdatgen_words_comb + "$")) & (df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráðu"
    df.loc[((df['sent3before'].str.match("^"+ acc_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráður"
    df.loc[((df['sent2before'].str.match("^"+ acc_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráður"
    df.loc[((df['sent2before'].str.match("^"+ dat_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráðum"
    df.loc[((df['sent3before'].str.match("^"+ dat_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráðum"
    df.loc[((df['sent3before'].str.match("^"+ gen_words + "$")) & (df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráða"
    df.loc[((df['sent2before'].str.match("^"+ gen_words + "$")) & (df['sentbefore'].str.match("^((\d{1,2}\.)?(\d{3}\.?)*(\d*[02-9]|\d+(,\d*[02-9]))|" + amounts + ")$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráða"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d*(1|(,\d*)1)$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráða"
    df.loc[((df['sentbefore'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*(\d+(,\d+)?|" + amounts + ")$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráður"
    df.loc[((df['sent2before'].str.match("^(\d{1,2}\.)?(\d{3}\.?)*\d+(,\d+)?$")) & (df['sentbefore'].str.match("^" + amounts + "$")) & (df['after_measure'].str.match("^°$"))), 'after_measure'] = "gráður"    


#### days

In [62]:
def df_days(df):
    print("df_days")
    df.loc[(df['after_measure'].str.match("^mán(ud)?\.?$")), 'after_measure'] = "mánudag"
    df.loc[(df['after_measure'].str.match("^þri(ðjud)?\.?$")), 'after_measure'] = "þriðjudag"
    df.loc[(df['after_measure'].str.match("^miðvikud?\.?$")), 'after_measure'] = "miðvikudag"
    df.loc[(df['after_measure'].str.match("^fim(mtud)?\.?$")), 'after_measure'] = "fimmtudag"
    df.loc[(df['after_measure'].str.match("^fös(tud)?\.?$")), 'after_measure'] = "föstudag"
    df.loc[(df['after_measure'].str.match("^lau(gard)?\.?$")), 'after_measure'] = "laugardag"
    df.loc[(df['after_measure'].str.match("^sun(nud)?\.?$")), 'after_measure'] = "sunnudag"


#### month

In [63]:
def df_months(df):
    print("df_months")
    df.loc[((df['after_measure'].str.match("^jan\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "janúar"
    df.loc[((df['after_measure'].str.match("^feb\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "febrúar"
    df.loc[((df['after_measure'].str.match("^mar\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "mars"
    df.loc[((df['after_measure'].str.match("^apr\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "apríl"
    df.loc[((df['after_measure'].str.match("^jún\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "júní"
    df.loc[((df['after_measure'].str.match("^júl\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "júlí"
    df.loc[((df['after_measure'].str.match("^ág\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "ágúst"
    df.loc[((df['after_measure'].str.match("^sept?\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "september"
    df.loc[((df['after_measure'].str.match("^okt\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "október"
    df.loc[((df['after_measure'].str.match("^nóv\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "nóvember"
    df.loc[((df['after_measure'].str.match("^des\.?$")) & (df['sentbefore'].str.match("^\d{1,2}\.$"))), 'after_measure'] = "desember"



#### roman numbers

In [64]:
def df_roman_numbers(df):
    print("df_roman_numbers")
    df.loc[(df['after_measure'].str.match("^II\.?$")), 'after_measure'] = "annar"
    df.loc[(df['after_measure'].str.match("^III\.?$")), 'after_measure'] = "þriðji"
    df.loc[(df['after_measure'].str.match("^IV\.?$")), 'after_measure'] = "fjórði"
    df.loc[(df['after_measure'].str.match("^VI\.?$")), 'after_measure'] = "sjötti"
    df.loc[(df['after_measure'].str.match("^VII\.?$")), 'after_measure'] = "sjöundi"
    df.loc[(df['after_measure'].str.match("^VIII\.?$")), 'after_measure'] = "áttundi"
    df.loc[(df['after_measure'].str.match("^IX\.?$")), 'after_measure'] = "níundi"
    df.loc[(df['after_measure'].str.match("^XI\.?$")), 'after_measure'] = "ellefti"
    df.loc[(df['after_measure'].str.match("^XII\.?$")), 'after_measure'] = "tólfti"
    df.loc[(df['after_measure'].str.match("^XIII\.?$")), 'after_measure'] = "þrettándi"
    df.loc[(df['after_measure'].str.match("^XIV\.?$")), 'after_measure'] = "fjórtándi"
    df.loc[(df['after_measure'].str.match("^XV\.?$")), 'after_measure'] = "fimmtándi"
    df.loc[(df['after_measure'].str.match("^XVI\.?$")), 'after_measure'] = "sextándi"
    df.loc[(df['after_measure'].str.match("^XVII\.?$")), 'after_measure'] = "sautjándi"
    df.loc[(df['after_measure'].str.match("^XVIII\.?$")), 'after_measure'] = "átjándi"
    df.loc[(df['after_measure'].str.match("^XIX\.?$")), 'after_measure'] = "nítjándi"


In [65]:
def main_measurement(df):
    start = time.time()
    df_tonn(df)
    df_hektari(df)
    df_somegram(df)
    df_somegram_noprior(df)
    df_gram(df)
    df_basegram(df)
    df_someliter(df)
    df_someliter_no_prior(df)
    df_somespoon(df)
    df_fet(df)
    df_inch(df)
    df_metri(df)
    df_somemeter(df)
    df_kwst(df)
    df_klst(df)
    df_sometime(df)
    df_crown(df)
    df_dollari(df)
    df_pound(df)
    df_yen(df)
    df_currencies(df)
    df_cur_letters(df)
    df_thousand_krona(df)
    df_million_krona(df)
    df_billion_krona(df)
    df_dimensionmeter_after(df)
    df_dimensionmeter_before(df)
    df_prefix_dimensionmeter_after(df)
    df_prefix_dimensionmeter_before(df)
    df_percent(df)
    df_stykki(df)
    df_electronic(df)
    df_watt(df)
    df_byte(df)
    df_cal(df)
    df_degree(df)
    df_days(df)
    df_months(df)
    df_roman_numbers(df)
    print(f"run done in {time.time() - start}")

In [ ]:
main_measurement(other_dataframe_expanded)

In [ ]:
df_comparison(other_dataframe_expanded, 'after', 'after_measure', 'measure')

In [ ]:
other_all_measures_type = other_dataframe_expanded[['sentence_id', 'token_id', 'before', 'after', 'after_measure', 'type']]

In [66]:
main_measurement(sport_dataframe_expanded)

df_tonn
df_hektari
df_somegram



df_somegram_noprior



df_gram
df_basegram



df_someliter



df_someliter_noprior


...


df_prefix_dimensionmeter_before



df_percent
df_stykki
df_electronic



df_watt



df_byte



df_cal
df_degree
df_days
df_months
df_roman_numbers
run done in 319.3626639842987


In [67]:
df_comparison(sport_dataframe_expanded, 'after', 'after_measure', 'measure')

df_comparison finished in 0.19866490364074707


### Tagger

In [ ]:
tokenize_nsw(other_all_measures_type, 'after_measure')

In [68]:
tokenize_nsw(sport_dataframe_expanded, 'after_measure')

length of dataframe is 13512



tokenize_nsw is done in 5.613346099853516


In [69]:
def set_type_second(df):
    start = time.time()
    df.loc[((df['after'].str.match(link_ptrn_all)) & (df['type'] == '')), 'type'] = "wlink"
    df.loc[((df['after'].str.match(ordinal_ptrn)) & (df['type'] == '')), 'type'] = "ordinal"
    print(f"set_type done in {time.time() - start}")

In [70]:
set_type_second(sport_dataframe_expanded)

set_type done in 0.41117000579833984


In [72]:
sport_dataframe_expanded.iloc[:5]

,sentence_id,token_id,before,after,type,sentbefore,sent2before,sent3before,sentafter,after_measure,compare
0,1,0,Áfram,Áfram,,,,,hélt,[Áfram],True
1,1,1,hélt,hélt,,Áfram,,,fjörið,[hélt],True
2,1,2,fjörið,fjörið,,hélt,Áfram,,í,[fjörið],True
3,1,3,í,í,,fjörið,hélt,Áfram,síðari,[í],True
4,1,4,síðari,síðari,,í,fjörið,hélt,hálfleik,[síðari],True


In [ ]:
set_type_second(other_all_measures_type)

In [73]:
sport_dataframe_expanded2 = flatten_nsw(sport_dataframe_expanded[['sentence_id', 'token_id', 'before', 'after_measure', 'type']])

flatten_nsw is done in 0.03196310997009277


In [ ]:
other_dataframe_expanded2 = flatten_nsw(other_all_measures_type[['sentence_id', 'token_id', 'before', 'after_measure', 'type']])



In [74]:
def get_sentences(df):
    start = time.time()
    uniquesents = df['sentence_id'].unique()
    
    allsents = []
    for sent in uniquesents:
        tmp_sent = df[df['sentence_id'] == sent]['after_measure'].tolist()
        allsents.append(tmp_sent)
    print(f"get_sentences done in {time.time() - start}")
    return allsents

In [75]:
def tag_into_df(df):
    start = time.time()
    allsents = get_sentences(df)
    tags = tagger.tag_bulk(allsents)
    flat_list_tags = [item for sublist in tags for item in sublist]
    df['tag'] = flat_list_tags
    df['next_tag'] = flat_list_tags[1:] + [""]
    print(f"tag_into_df done in {time.time() - start}")

In [76]:
tag_into_df(sport_dataframe_expanded2)

get_sentences done in 0.3118729591369629
tag_into_df done in 43.523003816604614


In [78]:
sport_dataframe_expanded2.iloc[:5]

,sentence_id,token_id,before,after_measure,type,tag,next_tag
0,1,0,Áfram,Áfram,,aa,sfg3eþ
1,1,1,hélt,hélt,,sfg3eþ,nheng
2,1,2,fjörið,fjörið,,nheng,af
3,1,3,í,í,,af,lkeþvm
4,1,4,síðari,síðari,,lkeþvm,nkeþ


In [ ]:
tag_into_df(other_dataframe_expanded2)

In [79]:
sport_df_save = sport_dataframe_expanded2[['sentence_id', 'token_id', 'after_measure', 'next_tag', 'type']]

In [ ]:
other_df_save = other_dataframe_expanded2[['sentence_id', 'token_id', 'after_measure', 'next_tag', 'type']]

In [80]:
sport_df_save.to_csv("sport_df_tagged_0503.csv")

In [ ]:
other_df_save.to_csv("other_df_tagged_0302.csv")